In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_teddynote.tools.tavily import TavilySearch
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage
import torch
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid

c:\ProgramData\miniconda3\envs\OoHMarketingSalse\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 불러오기
load_dotenv()

# tavily API 키 가져오기
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [3]:
class WebSearchState(TypedDict):
    context: Annotated[str, "Context"] # 요약   
    answer: Annotated[str, "Answer"] # 답변  
    messages: Annotated[list[BaseMessage], "Messages"] # 기록

In [4]:
def web_search(state: WebSearchState) -> WebSearchState:
    
    tavily_tool = TavilySearch()
    search_query = f"최근 한 달간 유행한 브랜드 이슈 또는 마케팅 사례"
    
    # 최근 한 달의 결과 5개 가져오기
    search_result = tavily_tool.search(
        query=search_query,
        topic="general",    # 'general'로 변경 (유효한 옵션)
        days=30,            # 최근 한 달 데이터로 확장
        max_results=3,      # 결과 수 증가
        format_output=True, # 결과 포맷팅
    )

    if search_result:
        formatted_result = f"## 최근 유행 브랜드 이슈 및 마케팅 사례 분석\n\n" + "\n\n".join(search_result)
        return WebSearchState(context=formatted_result)
    else:
        print("No search results found.")
        return WebSearchState(context="최근 브랜드 이슈에 대한 정보를 찾을 수 없습니다.")

In [ ]:
# 허깅페이스에서 한국어 생성형 모델 불러오는 함수
def get_huggingface_model(model_name: str = "skt/kogpt2-base-v2"):
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

# LLM 응답 생성 함수
def llm_answer(state: WebSearchState) -> WebSearchState:
    context = state["context"]

    report_prompt = f"""
    최근 트렌드가 되고 있는 브랜드 1개에 대해 다음 사항들을 중심으로 한국어로 상세한 분석 보고서를 작성해 주세요.

    형식:
    1. 최신 브랜드 10개 리스트 정리
    2. 각 브랜드의 시장에서의 강점과 특징
    3. 각 브랜드가 취하고 있는 마케팅 전략 및 포지셔닝
    4. 옥외 광고를 활용한 효과적인 마케팅 전략 및 제안

    최신 브랜드 이슈 정보:
    {context}
    """

    model, tokenizer = get_huggingface_model()

    # 프롬프트 토크나이즈 (길이 초과 방지)
    inputs = tokenizer(report_prompt, return_tensors="pt", truncation=True, max_length=2048)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return WebSearchState(
        answer=response,
        messages=[("assistant", response)]
    )

In [6]:
def makeGraph():
    # 워크 플로우   
    workflow = StateGraph(WebSearchState)

    workflow.add_node("web_search", web_search)
    workflow.add_node("llm_answer", llm_answer)

    workflow.set_entry_point("web_search")
    
    workflow.add_edge("web_search", "llm_answer")  
    workflow.add_edge("llm_answer", END)

    memory = MemorySaver()
    return workflow.compile(checkpointer=memory)

In [7]:
async def brand_analysis():

    app = makeGraph()
    config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})
    inputs = WebSearchState(
        context="",  
        answer="",   
        messages=[], 
        usage_metadata={}
    )

    result = await app.ainvoke(inputs, config)
    return result['answer'].content

In [8]:
import asyncio

async def main():
    result = await brand_analysis()
    print(result)

# notebook으로 실행 시
await main()

# py 파일로 실행 시
# if __name__ == "__main__":
#     asyncio.run(main())

IndexError: index out of range in self